In [ ]:
import numpy as np
import os
import pandas as pd
import psycopg

from dotenv import load_dotenv
from pathlib import Path

In [ ]:
dotenv_path = Path('pitcherlist_datascience.env')
load_dotenv(dotenv_path=dotenv_path)
print('Env loaded')

conn = psycopg.connect(user='postgres',#os.environ.get('PL_DB_USER'),
                       password='pacja0-sajKov-nahgiv',#os.environ.get('PL_DB_PASSWORD'),
                       host=os.environ.get('PL_DB_HOST'),
                       port="5432",
                       dbname="pl7")
print('Conn established')

cursor = conn.cursor()
print('Cursor established')

cursor.execute("Select * FROM pl_playerpositions")
colnames = [desc[0] for desc in cursor.description]
data = cursor.fetchall()

db_data = pd.DataFrame(data).copy()
db_data.columns = colnames

In [ ]:
# conn = psycopg.connect(user='postgres',#os.environ.get('PL_DB_USER'),
#                        password='pacja0-sajKov-nahgiv',#os.environ.get('PL_DB_PASSWORD'),
#                        host=os.environ.get('PL_DB_HOST'),
#                        port="5432",
#                        dbname="pl7")

# cursor = conn.cursor()

cursor.execute("SELECT * FROM players")
colnames = [desc[0] for desc in cursor.description]
data = cursor.fetchall()
id_data = pd.DataFrame(data).copy()
id_data.columns = colnames

In [ ]:
id_data.columns.values

In [ ]:
pos_df = pd.pivot(
    db_data.loc[db_data['game_year']==2025],
    index='mlb_player_id',
    columns='position',
    values=['games_played','games_started']
)[[('games_played', 'C'),('games_started', 'C'),
   ('games_played', '1B'),('games_started', '1B'),
   ('games_played', '2B'),('games_started', '2B'),
   ('games_played', '3B'),('games_started', '3B'),
   ('games_played', 'SS'),('games_started', 'SS'),
   ('games_played', 'MI'),('games_started', 'MI'),
   ('games_played', 'CI'),('games_started', 'CI'),
   ('games_played', 'OF'),('games_started', 'OF'),
   ('games_played', 'LF'),('games_started', 'LF'),
   ('games_played', 'CF'),('games_started', 'CF'),
   ('games_played', 'RF'),('games_started', 'RF')]].fillna(0).astype('int')

position_list = ['C','1B','2B','3B','SS','MI','CI','OF','LF','CF','RF']
high_level_list = ['C','1B','2B','3B','SS','OF']
pos_df.columns = [x for xs in [[x+'_p',x+'_s'] for x in position_list] for x in xs]

In [ ]:
for pos in position_list:
    pos_df[pos+'_elig'] = np.where((pos_df[pos+'_p']>=10) | (pos_df[pos+'_s']>=5),
                                   pos,
                                   None)
pos_df['pos_elig'] = pos_df[[x+'_elig' for x in high_level_list]].astype('str').agg('/'.join, axis=1).str.replace('/None','').str.replace('None/','').str.replace('None','UT')
pos_df

In [ ]:
pos_df.fillna(0).to_csv('2026_pos_eligibility.csv')